## 3. Using Discrete Fourier Transform (DFT) to Convert Between Coefficient and Value Forms

### Introduction

- So far, we know that the broad picture of FFT is as follows:
    1. We want to multiply 2 polynomials of degree $N$
    2. Doing this the regular way takes us $O(N^2)$ time
    3. But we can convert polynomials from coefficient form to value form in $O(N \log N)$ time using DFT
    4. Then multiply the polynomials together in $O(N)$ time
    5. Then convert back to coefficient form in $O(N \log N)$ time using inverse DFT

- In this section, we will focus on step 3, and get into the guts of the DFT implementation

### Convert from Coefficient to Value Form Using Recursion: Discrete Fourier Transform (DFT) 

- Let's assume we have the following polynomial

$$\begin{aligned}
    P(x) &= 3 - 5x + 2x^2 - 3x^3 + x^4
\end{aligned}$$

- So suppose we want to create the value form. Since this polynomial is of degree 4, we need to evaluate it at 5 different points 
    - Why 5 points? Because to re-derive the coefficients, you need to solve for 5 unknowns, and so you need 5 equations to solve for them

- Can we pick points that will make it more convenient for us to evaluate this equation? Yes we can! 
    - For example, let's suppose we're evaluating something like $P(x) = x^2$. Then we know that for any value $a$, $P(a) = P(-a)$
    - Functions that exhibit this behaviour are known as **even functions**
    - For polynomials, so long as they only have even degree terms (i.e. $P(x) = a + bx^2 + cx^4 + ...$ ), they are **even functions**

- So why is this useful? Remember we stated above that we need to evaluate $P(x)$ at 5 points. But if our function $P(x)$ happens to be an even function, evaluating it at any arbitrary point $a$ will also give us the value for the function as $-a$, effectively halving the number of evaluations we need to make!

- Ok, so that's all well and good. But $P(x) = 3 - 5x + 2x^2 - 3x^3 + x^4$ is not an even function, so how can we make use of this insight? 

- Turns out, any arbitrary polynomial $P(x)$ can be rewritten as a function of even functions! For example:

$$\begin{aligned}
    P(x) &= 3 - 5x + 2x^2 - 3x^3 + x^4 \\
    &= (3 + 2x^2 + x^4) + (-5x - 3x^3) \\
    &= (3 + 2x^2 + x^4) + x \cdot (-5 - 3x^2) \\
    &= P_E(x^2) + x \cdot P_O(x^2) \\ \\

    &\text{where} \\
    &\quad P_E(x^2) = 3 + 2x^2 + x^4 \\
    &\quad P_O(x^2) = -5 - 3x^2 \\
\end{aligned}$$

- Let's suppose we wish to evaluate $P(a)$ and $P(-a)$
    - Instead of simply calling the polynomial twice (which may be expensive), we can use the breakdown to compute $P_E(a^2)$ and $P_O(a^2)$
    - Then, we can evaluate $P(a) = $ and $P(-a)$ by taking
        - $P(a) = P_E(a^2) + a \cdot P_O(a^2)$
        - $P(-a) = P_E((-a)^2) - a \cdot P_O((-a)^2)$
    - The key point here is that, because $P_E$ and $P_O$ are even functions, $P_E(a^2) = P_E((-a)^2)$ and $P_O(a^2) = P_O((-a)^2)$

- Suppose we evaluate $P(x)$ at some arbitrary set of points $[-1, 1, -2, 2]$
    - Then we just need to evaluate $P_E, P_O$ at points $[1, 4]$. 
    - That is, we evaluate 2 functions at 2 points each, instead of 1 function at 4 points.
        - $P_E(1) = 6$, $P_E(4) = 27$, $P_O(1) = -8$, $P_O(4) = -17$
    - Then, to compute the 4 points required, we simply sum the existing results, instead of evaluating the function
        - $P(1) = 6 + -8 = -2$
        - $P(-1) = 6 - -8 = 14$
        - $P(2) = 27 + 2(-17) = -7$
        - $P(-2) = 27 - 2(-17) = 61$

- Naive algorithm: In the naive case of evaluating $N$ points for an $N$-degree polynomial, we compute $x^0, x^1 ..., x^N$ across $N$ values of $x$
    - Therefore, since we compute $N$ polynomials for $N$ different $x$ values, this ends up being $O(N^2)$
    - So converting from coefficient to value form naively takes $O(N^2)$

- DFT Algorithm:
    - For any $N$ points we wish to evaluate from polynomial $P(x)$, we can partition this into subproblems of $N/2$ points of 2 even sub-polynomials $P_E(x)$ and $P_O(x)$
    - Visualising this as a binary tree, this gives us a tree of height $\log_2(N)$
    - At each level of the tree, we still perform $N$ operations
        - We halve the number of points to evaluate as we go down each level
        - However, we are still computing the same number of powers of $x$ (i.e. Let each $x^0, x^1, etc.$ be 1 operation)
    - So this is why our time complexity is $O(N \log N)$ when converting from coefficient to value form

### DFT: Choosing points to evaluate

- We've seen DFT in action in the section above when evaluated on some arbitrary set of values $[-1,1,-2,2]$ above, and we claimed that the approach generalises to any arbitrary polynomial of degree $N$ But does it really? 

    - Let's see what happens when we actually need to do more than 1 recursive call by studying a polynomial of degree 7. For convenience, I'll show only the $P_E$ split for every recursion level

    $$\begin{aligned}
        P(x) &= 3 - 5x + 2x^2 - 3x^3 + x^4 + 2x^5 - x^6 + x^7 \\ \\

        P_{E_1}(x^2) &= 3 + 2x^2 + x^4 - x^6 \\
        &= 3 + 2x^2 + (x^2)^2 - (x^2)^3 \\ \\

        P_{E_2}(x^4) &= 3 + x^4 

    \end{aligned}$$

    - Let's suppose we wish to evaluate $P(x)$ on the same set of points $[-1,1,-2,2]$. Therefore
        - $P(x) => [-1,1,-2,2]$
        - $P_{E_1}(x^2) => [1, 4]$
        - $P_{E_2}(x^4) => [A]$ --> **WHAT SHOULD $A$ BE?** 

    - We know that we are halving the number of points we evaluate as we go down the recursive stack, which can be achieved by evaluating the sub-polynomials on points that map to 2 values in the stack above
        - That is, $P(-2)$ and $P(2)$ both make use of $P_{E_1}(4)$ in their evaluation

    - Therefore, by this logic, we need to choose $A$ such that $\sqrt{A} = 1$ is true, and $\sqrt{A} = 4$ is also true. Which, of course, is a blatent logical contradiction

- So we have a problem. DFT must be called recursively, but we can't seem to find a set of values that will allow us to meet the condition for recursion. How do we solve this contradiction?
    - Simple! Instead of using real numbers, we use **complex numbers** instead!
        - Let's say we want to evaluate $P(x)$ at $[-1, 1, -i, i]$ instead
        - Then, going one level down, we evaluate at $[1, -1]$
        - Then, going one more level down, we evaluate at $[1]$
        - This lets us do do recursion for a tree of height 3, one more level than before!


- Generalising this, if we have a tree of arbitrary height $\log(N)$, on which real or complex values should we evaluate the sub-polynomials?

- Turns out, the answer is to evaluate all solutions to the following equation, also known as the **roots of unity**

$$\begin{aligned}
    x^n &= 1
\end{aligned}

### DFT: Why evaluate the polynomials at the roots of unity?

- There is quite a lot of fundamentals that's needed here, so if you **really** have no time, you can just accept by rote that we choose to evaluate $P(x)$ at points where $x^n = 1$ to ensure that DFT works. But strongly suggest that you work through this, it's worth it

- Let's re-articulate the problem above
    - We need to pick $N$ different values of $x$, and for each point evaluate $P(x)$. This will give us the value form of $P(x)$
    - We also showed that these values of $x$ cannot be real; they must be complex

- Any complex value can be expressed as the sum of its real and complex parts. 
$$\begin{aligned}
    \zeta &= a + i b
\end{aligned}$$

- And any complex value can be rewritten in polar form, if we imagine the real component being on the x-axis and the imaginary component on the y-axis
$$\begin{aligned}
    \zeta &= a + i b \\
    &= r (\cos \theta + i \cdot \sin \theta) \\
    &= r e^{i \theta} \qquad \because e^{i \theta} = \cos \theta + i \cdot \sin \theta
\end{aligned}$$

- Let $\omega$ be the v

Then, we want to pick $\zeta$ such that $\zeta^{\frac{1}{n}}$ gives us real values
$$\begin{aligned}
    \zeta^{\frac{1}{n}} &= r^{\frac{1}{n}} e^{\frac{i \theta}{n}}
\end{aligned}$$

- Now, in theory, 